In [7]:
import cv2
import numpy as np
from skimage.feature import peak_local_max
from scipy.ndimage import label

def process_image1(image):
    # Load YOLO
    net = cv2.dnn.readNet("./darknet/yolov3.weights", "./darknet/cfg/yolov3.cfg")

    # Load class labels
    classes = []
    with open("./darknet/data/coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]

    # Get the image dimensions
    height, width, _ = image.shape

    # Preprocess the image
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)

    # Set the input blob
    net.setInput(blob)

    # Forward pass through the network
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    outputs = net.forward(output_layers)

    # Extract bounding box coordinates and class predictions
    boxes = []
    confidences = []
    class_ids = []
    labels = []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w/2)
                y = int(center_y - h/2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                labels.append(classes[class_id])

    # Apply non-maximum suppression to eliminate overlapping bounding boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw bounding boxes and labels
    for i in indices:
        # i = i[0]
        x, y, w, h = boxes[i]
        label = labels[i]
        confidence = confidences[i]
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(image, f"{label}: {confidence:.2f}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return image, labels
def perform_canny_edge_detection(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Return the edges image
    return edges

def apply_feature_selection(edge_image, min_distance=10, threshold_abs=5, area_threshold=50):
    # Find local peaks in the edge image
    coordinates = peak_local_max(edge_image, min_distance=min_distance, threshold_abs=threshold_abs)
    
    # Create a binary mask for the detected features
    features_mask = np.zeros_like(edge_image, dtype=bool)
    features_mask[tuple(coordinates.T)] = True
    
    # Label connected regions in the features mask
    labeled_features, num_features = label(features_mask)
    
    # Filter out small regions based on their area
    for feature_label in range(1, num_features + 1):
        area = np.sum(labeled_features == feature_label)
        if area < area_threshold:
            labeled_features[labeled_features == feature_label] = 0
    
        # Convert the labeled features back to an image
    selected_features_image = labeled_features.astype(np.uint8) * 255
    
    # Return the selected features image
    return selected_features_image
# Test the function
# input_image = cv2.imread("./image.jpg")
# output_image, detected_labels = process_image1(input_image)

# # Display the output image
# cv2.imshow("Object Detection", output_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# # Print the detected labels
# print("Detected Labels:")
# for label in detected_labels:
#     print(label)
###################################
# image=cv2.imread("./test.jpg")
# edges = perform_canny_edge_detection(image)
# #show the original image and the edge detected image
# selected= apply_feature_selection(edges)
# # print(selected)
# cv2.imshow("Original Image", image)
# cv2.imshow("Edge Detected Image", edges)
# cv2.imshow("Selected Features", selected)
# # cv2.imshow("Selected Features", selected.astype(np.uint8) * 255)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [8]:
from flask import Flask, request, send_file, jsonify
import base64
from PIL import Image
import io
import numpy as np
import cv2
import tensorflow as tf
# from tensorflow.keras.applications import EfficientDetModel
app = Flask(__name__)

# model = tf.keras.applications.MobileNetV2(weights='imagenet')
# model = tf.keras.applications.EfficientNetB0(weights='imagenet')
def detect_edges(img):
    edges = cv2.Canny(img, 100, 200)
    return edges


@app.route('/process-image1', methods=['POST'])
def process_image_route():
    image_file = request.files.get('image')
    image = cv2.imdecode(np.frombuffer(image_file.read(), np.uint8), cv2.IMREAD_COLOR)
    image1 = image.copy()

    # Process the image
    processed_image, detected_labels = process_image1(image1)

    # Convert the processed image to JPEG format
    ret, jpeg = cv2.imencode('.jpg', processed_image)
    encoded_image = jpeg.tobytes()

    # Create a response JSON object
    response = {
        'labels': detected_labels,
        'image': base64.b64encode(encoded_image).decode('utf-8')
    }

    # Return the response as JSON
    return jsonify(response)

    # Return the encoded image as a response
    # return Response(encoded_image, mimetype='image/jpeg')
if __name__ == '__main__':
    #run app in debug mode on port 5000
    app.run(host='0.0.0.0', port=5000)
    


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.7:5000/ (Press CTRL+C to quit)
